In [1]:
dayObs = 'Today'
run = "None"

In [2]:
from lsst.daf.butler import Butler
from astropy.table import Table
from astropy.time import Time, TimeDelta
import tabulate

In [3]:
if dayObs=='Today':
    dayObs = (Time.now() - TimeDelta(0.5, format='jd')).iso[:10].replace('-','')
dayObs

'20241013'

In [4]:
butler = Butler('embargo_new', collections=[
    'LSSTCam/raw/all',
    'LSSTCam/calib/unbounded',
    'LSSTCam/photodiode'
])

In [5]:
exposureList = []

if run is "None":
    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"
else:
    where=f"exposure.science_program={run} and instrument='LSSTCam'"    

for record in butler.registry.queryDimensionRecords("exposure", where=where):
    nimage=len(list(butler.registry.queryDatasets('raw',dataId=record.dataId)))
    hasPD=len(list(butler.registry.queryDatasets('photodiode',dataId=record.dataId)))
    exposureList.append([record.id, record, nimage, hasPD])
exposureList.sort(key=lambda x: x[0])
table = Table(rows=[ (
            f'<a href=http://lsstcam-mcm.cp.lsst.org/FITSInfo/view.html?image={record.obs_id})>{record.obs_id}</a>',
            record.timespan.begin.iso,
            record.seq_num,
            record.observation_type,
            record.science_program,
            record.observation_reason,
            record.exposure_time,
            record.physical_filter,
            nimage,
            hasPD,            
                     )
            for [id,record,nimage, hasPD] in exposureList ]
            )


In [6]:
#table.pformat_all()
tabulate.tabulate(table, tablefmt='unsafehtml',headers=[
                   'obs_id', 'time', 'seqnum', 'observation_type',
                   'science_program', 'observation_reason','exposure_time',
                   'physical_filter', 'nimage', 'hasPD'
                  ])

obs_id,time,seqnum,observation_type,science_program,observation_reason,exposure_time,physical_filter,nimage,hasPD
MC_C_20241013_000001,2024-10-13 12:28:14.578014,1,bias,E756,preimage,0,unknown,197,0
MC_C_20241013_000002,2024-10-13 12:28:17.169994,2,bias,E756,preimage,0,unknown,197,0
MC_C_20241013_000003,2024-10-13 12:28:20.108984,3,bias,E756,preimage,0,unknown,197,0
MC_C_20241013_000004,2024-10-13 12:28:23.070988,4,bias,E756,bias,0,unknown,197,0
MC_C_20241013_000005,2024-10-13 12:28:26.066988,5,bias,E756,bias,0,unknown,197,0
MC_C_20241013_000006,2024-10-13 12:28:28.889987,6,bias,E756,bias,0,unknown,197,0
MC_C_20241013_000007,2024-10-13 12:28:31.486995,7,bias,E756,bias,0,unknown,197,0
MC_C_20241013_000008,2024-10-13 12:28:34.487985,8,bias,E756,bias,0,unknown,197,0
MC_C_20241013_000009,2024-10-13 12:28:37.368999,9,bias,E756,bias,0,unknown,197,0
MC_C_20241013_000010,2024-10-13 12:28:40.250013,10,bias,E756,bias,0,unknown,197,0
